In [1]:
from pymongo import MongoClient

In [3]:
client=MongoClient()
client=MongoClient('mongodb://localhost:29017/')
db=client['eventData']
sen=db.documents_english

In [4]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

In [5]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

since sen.find(0 cursor run the order never change so we can get the docId attached to the training result in this way.

In [8]:
%%time
texts = []
docIds=[]
actuallyTrained=0;
temp=0;
for i in sen.find():
    if temp<1000:
        temp=temp+1
        try:
            raw = ''.join(i['document']).lower()
            tokens = tokenizer.tokenize(raw)
            stopped_tokens = [i for i in tokens if not i in en_stop]
            stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
            texts.append(stemmed_tokens)
            docIds.append(i['_id'])
            actuallyTrained=actuallyTrained+1
        except:
            pass
    else:
        break
print(actuallyTrained)

1000
CPU times: user 3.46 s, sys: 20 ms, total: 3.48 s
Wall time: 3.49 s


In [9]:
%%time
dictionary = corpora.Dictionary(texts)

CPU times: user 156 ms, sys: 0 ns, total: 156 ms
Wall time: 154 ms


In [10]:
%%time
corpus = [dictionary.doc2bow(text) for text in texts]

CPU times: user 204 ms, sys: 0 ns, total: 204 ms
Wall time: 203 ms


In [11]:
%%time
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=20, id2word = dictionary, passes=1)

CPU times: user 17.1 s, sys: 2.09 s, total: 19.2 s
Wall time: 16.9 s


In [12]:
actuallyTrained

1000

#change result into multidimensinal array in order to feed in k-means model,
#the no of dimension is the no of topics

In [13]:
%%time
#the dim is the same number of topics
dim=20 
result=[]
for i in range(0,actuallyTrained):
    feature=[]
    previousindex=0
    for item in ldamodel[corpus[i]]:
        index=item[0]
        #print(index)
        for beforeindex in range(previousindex,index):
            feature.append(0)
        feature.append(item[1])
        previousindex=index+1
    while (len(feature)<dim):
        feature.append(0);  #add in 0 at the end
    result.append(feature)

CPU times: user 1.97 s, sys: 1.44 s, total: 3.41 s
Wall time: 1.81 s


In [14]:
from sklearn.cluster import KMeans
import numpy as np

In [15]:
%%time
kmeanstest=np.array(result)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.82 ms


In [16]:
%%time
kmeans = KMeans(n_clusters=20, random_state=0).fit(kmeanstest)

CPU times: user 536 ms, sys: 3.66 s, total: 4.2 s
Wall time: 305 ms


In [17]:
kmeans.labels_.size

1000

In [18]:
len(docIds)

1000

#build a dictionary for ['docId','cluster #']

In [19]:
#and before building  the dictionary test if the size of docIds and cluster result dimensions are the same.
try:
    assert(len(docIds)==kmeans.labels_.size)
    dictionary_cocId_topicClusterItBelongs={}
    for i in range(0,actuallyTrained):
        dictionary_cocId_topicClusterItBelongs.update({docIds[i]:kmeans.labels_[i]})
except:
    print("the docIds size is different from the topic # cluster size")
    

In [20]:
dictionary_cocId_topicClusterItBelongs

{'56ef2cdf172ab8077b9975de': 4,
 '56ef2cdf172ab8077b9975e4': 16,
 '56ef2cdf172ab8077b9975e0': 8,
 '56ef2c88172ab806913ca82b': 11,
 '56ef2c89172ab806943ca85b': 9,
 '56ef2cdd172ab8077b99758f': 1,
 '56ef2c88172ab806923ca847': 0,
 '56ef2cde172ab8077a9975c3': 4,
 '56ef2c87172ab806903ca800': 4,
 '56ef2cdf172ab807789975f7': 14,
 '56ef2c87172ab806923ca81a': 1,
 '56ef2cde172ab807789975c0': 11,
 '56ef2c87172ab806933ca80a': 1,
 '56ef2cdd172ab80779997595': 3,
 '56ef2cdf172ab8077a997606': 0,
 '56ef2c87172ab806933ca80b': 9,
 '56ef2c88172ab806913ca83e': 4,
 '56ef2cde172ab807799975db': 6,
 '56ef2c87172ab806933ca80d': 16,
 '56ef2c88172ab806943ca84a': 1,
 '56ef2c89172ab806903ca8a0': 6,
 '56ef2c8a172ab806903ca8b4': 19,
 '56ef2c87172ab806913ca81d': 8,
 '56ef2c88172ab806943ca84f': 3,
 '56ef2cde172ab8077a9975c1': 18,
 '56ef2cde172ab807789975da': 0,
 '56ef2c88172ab806943ca823': 19,
 '56ef2c88172ab806903ca85f': 0,
 '56ef2c87172ab806913ca81b': 1,
 '56ef2cde172ab807799975cd': 4,
 '56ef2c89172ab806923ca869': 4,


In [23]:
#using pickle to dump and load the data
import pickle

In [25]:
with open('traingrst_english.pkl', 'wb') as output:
    pickle.dump(dictionary_cocId_topicClusterItBelongs,output)